# Dog Breed Classification Model 🐶

This notebook builds an end-to-end multi-class image classifier using TensorFlow 2 and TensorFlow Hub.

## 1. Problem

Identifying the breed of a dog given an image of a dog.

## 2. Data

The data is from Kaggle's dog breed identification competition.

https://www.kaggle.com/competitions/dog-breed-identification/data

## 3. Evaluation

A file with prediction probabilities of each dog breed for each test image.

## 4. Features

Info about the data:
* Using unstructured data (images) so will be best to use deep learning/transfer learning.
* There are 120 breeds meaning, 120 different classes.
* There are 10,000+ images in the training set (with labels)
* There are 10,000+ images in the test set (no labels) 

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf


In [12]:
# Check for GPU availability 
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")


GPU available
